In [47]:
import datetime
import random
import json

In [83]:
question_format = """A and B want to schedule a {interval}-hour meeting together.
A's availability: {A_availability}
B's availability: {B_availability}
What time slot works best? (if multiple, choose the earliest one)"""

In [124]:
def get_sample():

    interval_list = [0.5, 1, 1.5]

    all_slots = [8 + 0.5 * x for x in range(0, 20)]
    A_availability_size = random.randint(5, 15)
    B_availability_size = random.randint(5, 15)
    # sample without replacement
    A_availability = random.sample(all_slots, A_availability_size)
    B_availability = random.sample(all_slots, B_availability_size)

    # Convert A_availability and B_availability to strings in the format of "hh:mm-hh:mm" and merge adjacent time slots
    def convert_to_string(availability):
        availability.sort()
        current_start = availability[0]
        current_end = availability[0] + 0.5
        strings = []
        for i in range(1, len(availability)):
            if availability[i] == current_end:
                current_end += 0.5
            else:
                # Convert current_start and current_end to string
                current_start = str(int(current_start)).zfill(2) + ":" + str(int((current_start - int(current_start)) * 60)).zfill(2)
                current_end = str(int(current_end)).zfill(2) + ":" + str(int((current_end - int(current_end)) * 60)).zfill(2)
                strings.append(current_start + " - " + current_end)
                current_start = availability[i]
                current_end = availability[i] + 0.5
        # Convert current_start and current_end to string
        current_start = str(int(current_start)).zfill(2) + ":" + str(int((current_start - int(current_start)) * 60)).zfill(2)
        current_end = str(int(current_end)).zfill(2) + ":" + str(int((current_end - int(current_end)) * 60)).zfill(2)
        strings.append(current_start + " - " + current_end)

        return ", ".join(strings)

    A_availability_string = convert_to_string(A_availability)
    B_availability_string = convert_to_string(B_availability)

    # Generate random interval
    interval = random.choice(interval_list)

    # Generate question
    question = question_format.format(interval=interval, A_availability=A_availability_string, B_availability=B_availability_string)

    # print(question)

    # Generate answer
    overlap = sorted(list(set(A_availability) & set(B_availability)))
    answer = "No time slot works."
    if len(overlap) != 0:
        current_start = overlap[0]
        current_end = overlap[0] + 0.5
        for i in range(1, len(overlap)):
            if overlap[i] == current_end:
                current_end += 0.5
            else:
                if current_end - current_start >= interval:
                    current_end = current_start + interval
                    current_start = str(int(current_start)).zfill(2) + ":" + str(int((current_start - int(current_start)) * 60)).zfill(2)
                    current_end = str(int(current_end)).zfill(2) + ":" + str(int((current_end - int(current_end)) * 60)).zfill(2)
                    answer = current_start + " - " + current_end
                    break
                current_start = overlap[i]
                current_end = overlap[i] + 0.5
        if answer == "No time slot works." and current_end - current_start >= interval:
            current_end = current_start + interval
            current_start = str(int(current_start)).zfill(2) + ":" + str(int((current_start - int(current_start)) * 60)).zfill(2)
            current_end = str(int(current_end)).zfill(2) + ":" + str(int((current_end - int(current_end)) * 60)).zfill(2)
            answer = current_start + " - " + current_end
    # print(answer)
    sample = {"input": question, "target": answer}
    return sample

In [139]:
task = {
    "examples": [get_sample() for i in range(250)]
}

In [140]:
from pprint import pprint
pprint(get_sample())

{'input': 'A and B want to schedule a 1.5-hour meeting together.\n'
          "A's availability: 08:00 - 10:00, 11:00 - 11:30, 12:30 - 15:30, "
          '16:30 - 18:00\n'
          "B's availability: 09:00 - 09:30, 10:00 - 11:00, 12:00 - 12:30, "
          '13:00 - 14:00, 14:30 - 15:30, 16:00 - 18:00\n'
          'What time slot works best? (if multiple, choose the earliest one',
 'target': '16:30 - 18:00'}


In [141]:
with open("bbh/schedule_meeting.json", "w") as f:
    json.dump(task, f, indent=4)

In [136]:
demos = [get_sample() for i in range(3)]

cot_prompt = f"""
-----
Given two people's availability, find the first time slot that works best for both of them.

Q: {demos[0]["input"]}
A: {demos[0]["target"]}

Q: {demos[1]["input"]}
A: {demos[1]["target"]}

Q: {demos[2]["input"]}
A: {demos[2]["target"]}"""

In [137]:
print(cot_prompt)


-----
Given two people's availability, find the first time slot that works best for both of them.

Q: A and B want to schedule a 1.5-hour meeting together.
A's availability: 08:00 - 09:00, 10:30 - 12:00, 13:30 - 14:00, 15:00 - 15:30, 16:00 - 17:00
B's availability: 08:00 - 08:30, 10:00 - 10:30, 14:00 - 15:30, 16:00 - 17:00, 17:30 - 18:00
What time slot works best? (if multiple, choose the earliest one
A: No time slot works.

Q: A and B want to schedule a 1.5-hour meeting together.
A's availability: 08:30 - 09:00, 10:00 - 11:30, 13:00 - 13:30, 16:00 - 16:30
B's availability: 08:30 - 09:00, 12:30 - 13:30, 15:00 - 16:30
What time slot works best? (if multiple, choose the earliest one
A: No time slot works.

Q: A and B want to schedule a 0.5-hour meeting together.
A's availability: 08:00 - 09:00, 10:00 - 11:30, 12:30 - 13:00, 16:30 - 17:30
B's availability: 08:00 - 09:30, 10:00 - 11:00, 11:30 - 12:00, 12:30 - 14:00, 14:30 - 17:00, 17:30 - 18:00
What time slot works best? (if multiple, cho

In [138]:
with open("cot-prompts/schedule_meeting.txt", "w") as f:
    f.write(cot_prompt)